In [ ]:
# %pylab nbagg
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

from lab_repo.classes.dbclasses import dbExperimentSet, dbExperiment
from lab_repo.classes.classes import pcExperimentGroup

import numpy as np
import scipy
from scipy.stats import pearsonr, binned_statistic, sem, f_oneway, ttest_1samp, ttest_ind, ttest_rel, wilcoxon, mannwhitneyu
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import pandas as pd

from datetime import datetime

import lab_repo.analysis.dendrites_analysis as da
from lab_repo.misc.plotting import cdf


import cPickle as pkl
import os

import lab_repo.misc.plotting as lpl

from matplotlib import rc

import lab_repo.analysis.place_cell_analysis as pca

# Set rc-params
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 7
plt.rcParams['boxplot.boxprops.linewidth'] = 1

rc('font',**{'family':'sans-serif','sans-serif':['helvetica', 'sans-serif']})
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
def initializeExperimentGroup():

    pc_kwargs = {'imaging_label': 'mergedmerged',
                 'nPositionBins': 100,
                 'channel': 'Ch2',
                 'demixed': False,
                 'pf_subset': None,
                 'signal': None}

    path = '/home/sebi/grps/dend_grp.json'
    eset = dbExperimentSet(project_name='sebi')
    grp = pcExperimentGroup.from_json(path, eset, **pc_kwargs)

    return grp

def group_order(x):
    
    if x['order'] < 7:
        return x['order']
    else:
        return 7
    
def clean(ax, full=False, offset=0):
    if full:
        sns.despine(ax=ax, left=True, bottom=True, offset=offset)
        ax.set_yticks([])
        ax.set_xticks([])
    else:
        sns.despine(ax=ax, offset=offset)

In [ ]:
grp = initializeExperimentGroup()
grp = pcExperimentGroup([x for x in grp if x.condition==1], **grp.args)
colors = sns.xkcd_palette(['red orange', 'black', 'purple'])

## Plotting

### Tuning Curve Correlations

In [ ]:
plotdir = '/home/sebi/plots/dends/fig3'

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

df = tc_corrs.groupby(['expt', 'soma']).mean()
cdf(ax, df.loc[df['pf'] == 0].dropna()['corr'], color='0.5', lw=1)
cdf(ax, df.loc[df['pf'] == 1].dropna()['corr'], color='orange', lw=1)

clean(ax, offset=4)
ax.set_xlabel('Mean Dendrite-Soma\nTuning Correlation')
ax.set_ylabel('Cumulative Fraction')

left, bottom, width, height = [0.7, 0.125, 0.2, 0.2]
ax2 = fig.add_axes([left, bottom, width, height])

sns.barplot(y='corr', x='pf', palette=['0.5', 'orange'], data=df, ci=68, capsize=0.1, errwidth=1,
            dodge=False, edgecolor='0.2', linewidth=1, ax=ax2)
clean(ax2, offset=1)
ax2.set_ylabel('')
ax2.set_xlabel('')
ax2.set_xticks([])
ax2.set_yticks([0, 0.5])
ax2.set_yticklabels(['0', '0.5']);

# fig.savefig(os.path.join(plotdir, 'tc_corrs.svg'))

In [ ]:
mannwhitneyu(df.loc[df['pf'] == 0].dropna()['corr'], df.loc[df['pf'] == 1].dropna()['corr'])

In [ ]:
# Is this just due to increased transient sizes?
run_bsp = pd.read_pickle(os.path.join(data_path, 'rf_running_bsp.pkl'))

In [ ]:
mean_bsp = run_bsp.groupby(['roi', 'expt'], as_index=False).mean()
mean_corr = tc_corrs.groupby(['expt', 'soma'], as_index=False).mean()
merge_df = mean_bsp.merge(mean_corr, left_on=['roi', 'expt'], right_on=['soma', 'expt'])

In [ ]:
fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

min_bin = np.nanpercentile(merge_df['amplitude'], 1)
max_bin = np.nanpercentile(merge_df['amplitude'], 99)

n_bins=7

df = merge_df.loc[merge_df['pf'] == 1]
df = df.dropna(subset=['amplitude', 'corr'])

color = 'orange'
means, bins, _ = binned_statistic(df['amplitude'], df['corr'],
                                  statistic=np.nanmean,
                                  bins=np.logspace(np.log10(min_bin),np.log10(max_bin), n_bins))

stds, bins, _ = binned_statistic(df['amplitude'], df['corr'],
                                  statistic=sem,
                                  bins=np.logspace(np.log10(min_bin),np.log10(max_bin), n_bins))
                
log_bins = np.log10(bins)
plot_bins = 10 ** (log_bins[:-1] + 0.5 * (log_bins[1] - log_bins[0]))

ax.errorbar(np.log10(plot_bins * 100), means, yerr=stds, fmt='o', mfc=color,
            mec=color, ecolor=color, ms=1, elinewidth=1, mew=0.5,
            ls='-', color=color, lw=1)

df = merge_df.loc[merge_df['pf'] == 0]
df = df.dropna(subset=['amplitude', 'corr'])
color = '0.5'
means, bins, _ = binned_statistic(df['amplitude'], df['corr'],
                                  statistic=np.nanmean,
                                  bins=np.logspace(np.log10(min_bin),np.log10(max_bin), n_bins))

stds, bins, _ = binned_statistic(df['amplitude'], df['corr'],
                                  statistic=sem,
                                  bins=np.logspace(np.log10(min_bin),np.log10(max_bin), n_bins))

log_bins = np.log10(bins)
plot_bins = 10 ** (log_bins[:-1] + 0.5 * (log_bins[1] - log_bins[0]))

ax.errorbar(np.log10(plot_bins * 100), means, yerr=stds, fmt='o', mfc=color,
            mec=color, ecolor=color, ms=1, elinewidth=1, mew=0.5,
            ls='-', color=color, lw=1, zorder=1000)

clean(ax, offset=4)
ax.set_xlabel(r'Mean Somatic %$\Delta$F/F')
ax.set_ylabel('Dendrite-Soma Tuning Correlation')

ax.set_xticks([2, 3])
ax.set_xticklabels([100, 1000])
ax.set_yticks(np.arange(0, 1.2, 0.2))
# sns.boxplot(y='amplitude', x='pf', data=merge_df, whis=[5, 95], showfliers=False)
# fig.savefig(os.path.join(plotdir, 'tc_corr_by_amp.svg'))

In [ ]:
from statsmodels.formula.api import ols
formula = 'corr ~ amplitude + C(pf) + C(pf)*amplitude'
lm = ols(formula, merge_df).fit()

In [ ]:
lm.pvalues

In [ ]:
lm.summary()

In [ ]:
def pf_type(row):
    
    pfo = row['pf_overlap']
    
    if np.isnan(pfo):
        return 'Soma'
    elif pfo:
        return 'Co-PF'
    else:
        return 'Iso-PF'

In [ ]:
all_freq = pd.read_pickle(os.path.join(data_path, 'rf_all_freq.pkl'))
run_freq = pd.read_pickle(os.path.join(data_path, 'rf_run_freq.pkl'))

In [ ]:
run_freq = run_freq.rename(index=str, columns={'roi':'dend'})

In [ ]:
# sens = sens.rename(index=str, columns={'expt_id': 'expt', 'roi':'dend', 'FOV':'fov'})
# spef = spef.rename(index=str, columns={'expt_id': 'expt', 'roi':'dend', 'FOV':'fov'})
# split_corr = split_corr.rename(index=str, columns={'expt_id': 'expt', 'roi':'dend', 'FOV':'fov'})
merge_df = sens.merge(spef, on=['expt', 'fov', 'dend'], suffixes=['_sens', '_spef'])
merge_df = merge_df.merge(run_freq, on=['expt', 'fov', 'dend'], suffixes=['', '_split'])
merge_df = merge_df.merge(tc_corrs, on=['expt', 'fov', 'dend'], suffixes=['', '_corr'], how='left')

merge_df['pf_type'] = merge_df.apply(pf_type, axis=1)

cols = ['value_spef', 'value_sens', 'frequency', 'order']
names = ['Specificity', 'Sensitivity', 'RUN Event Rate (Hz)', 'Branch Order']

colors = ['#ec5e34', '#2bb673', '#2278b5']

for i in xrange(1, 4):
    fig = plt.figure(figsize=(1, 1.5), dpi=200)
    ax = fig.add_subplot(111)
    sns.boxplot(x='pf_type', y=cols[i-1], data=merge_df, whis=[5, 95], showfliers=False,
                order=['Iso-PF', 'Co-PF', 'Soma'], palette=colors)
    plt.setp(ax.artists, edgecolor='k', lw=1);
    plt.setp(ax.lines, color='k', lw=1);
    sns.stripplot(x='pf_type', y=cols[i-1], data=merge_df, edgecolor='k', linewidth=0.5, alpha=0.5,
                  dodge=True, size=2, order=['Iso-PF', 'Co-PF', 'Soma'], palette=colors)
    clean(ax, offset=4)

    ax.set_xlabel('')
    ax.set_ylabel(names[i-1])
    if i in [1, 2]:
        ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.xticks(rotation=45)
    
    fig.savefig(os.path.join(plotdir, 'iso_pf_tuning_quality_{}.svg'.format(cols[i-1])))

In [ ]:
col = 'value_spef'

a = merge_df.loc[merge_df['pf_type'] == 'Iso-PF'][col].dropna()
b = merge_df.loc[merge_df['pf_type'] == 'Co-PF'][col].dropna()
c = merge_df.loc[merge_df['pf_type'] == 'Soma'][col].dropna()

print f_oneway(a, b, c)

vals = merge_df.dropna(subset=[col])

print pairwise_tukeyhsd(endog=vals[col], groups=vals['pf_type'], alpha=0.05)

In [ ]:
col = 'value_sens'

a = merge_df.loc[merge_df['pf_type'] == 'Iso-PF'][col].dropna()
b = merge_df.loc[merge_df['pf_type'] == 'Co-PF'][col].dropna()
c = merge_df.loc[merge_df['pf_type'] == 'Soma'][col].dropna()

print f_oneway(a, b, c)

vals = merge_df.dropna(subset=[col])

print pairwise_tukeyhsd(endog=vals[col], groups=vals['pf_type'], alpha=0.05)

In [ ]:
col = 'frequency'

a = merge_df.loc[merge_df['pf_type'] == 'Iso-PF'][col].dropna()
b = merge_df.loc[merge_df['pf_type'] == 'Co-PF'][col].dropna()
c = merge_df.loc[merge_df['pf_type'] == 'Soma'][col].dropna()

print f_oneway(a, b, c)

vals = merge_df.dropna(subset=[col])

print pairwise_tukeyhsd(endog=vals[col], groups=vals['pf_type'], alpha=0.05)

In [ ]:
fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

merge_df['abs_peak_dist'] = merge_df['peak_dist'].apply(np.abs)

lpl.cdf(ax, merge_df.loc[(merge_df['pf_type']=='Co-PF') & merge_df['pf']]['abs_peak_dist'], color=colors[1], lw=1);
# lpl.cdf(ax, np.abs(merge_df.loc[(merge_df['pf_type']=='Iso-PF') & merge_df['pf']]['peak_dist']), color=colors[0], lw=1);
# lpl.cdf(ax, np.abs(merge_df.loc[(merge_df['pf_type']=='Iso-PF') & (merge_df['pf']==False)]['peak_dist']), color='orange', lw=1);

lpl.cdf(ax, merge_df.loc[(merge_df['pf_type']=='Iso-PF')]['abs_peak_dist'], color=colors[0], lw=1);

clean(ax, offset=4)
ax.set_xlim([0, 100])
ax.set_xlabel('Distance to Soma Peak (cm)')
ax.set_ylabel('Cumulative Fraction')

left, bottom, width, height = [0.7, 0.125, 0.2, 0.2]
ax2 = fig.add_axes([left, bottom, width, height])

sns.barplot(y='abs_peak_dist', x='pf_type', palette=[colors[1], colors[0]], order=['Co-PF', 'Iso-PF'], data=merge_df, ci=68, capsize=0.1, errwidth=1,
            dodge=False, edgecolor='0.2', linewidth=1, ax=ax2)
clean(ax2, offset=1)
# ax2.set_ylabel('Fraction Coactive')
ax2.set_ylabel('')
ax2.set_xlabel('')
ax2.set_xticks([])
ax2.set_ylim([0, 55])
ax2.set_yticks([0, 25, 50])
# ax2.set_yticks([0, 0.5])
# ax2.set_yticklabels(['0', '0.5']);

# fig.savefig(os.path.join(plotdir, 'dend_pf_tuning_dist.svg'))

In [ ]:
wilcoxon(merge_df.loc[merge_df['pf_type'] == 'Iso-PF']['abs_peak_dist'] - 48)

### BSP stability

In [ ]:
in_pf_bsp = da.branch_spike_prevalence(grp, signal=None, running_only=True, in_field_only=True)

In [ ]:
# in_pf_bsp = pd.read_pickle(os.path.join(data_path, 'rf_in_pf_bsp.pkl'))

colors = sns.xkcd_palette(['reddish purple', 'pumpkin'])

mean_bsp = in_pf_bsp.groupby(['expt', 'roi', 'mouse_name', 'fov'], as_index=False).mean()
merge_df = mean_bsp.merge(paired_activity_dists, on=['expt', 'roi'])

fig = plt.figure(figsize=(1.5, 1.5), dpi=200)
ax = fig.add_subplot(111)

df = merge_df.dropna(subset=['absolute_centroid_shift', 'value'])
df1 = df.loc[df['day_1'] == 1]
df1['stable'] = df1['absolute_centroid_shift'] < df1['absolute_centroid_shift'].median()
df2 = df.loc[df['day_1'] == 2]
df2['stable'] = df2['absolute_centroid_shift'] < df2['absolute_centroid_shift'].median()

df = pd.concat([df1, df2])

# colors = sns.xkcd_palette(['cobalt', 'emerald'])
colors = sns.xkcd_palette(['reddish purple', 'pumpkin'])
sns.boxplot(x='day_1', y='value', hue='stable', data=df, whis=[5, 95], showfliers=False, palette=colors)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='day_1', y='value', hue='stable', data=df, edgecolor='k', linewidth=0.5, alpha=0.5, dodge=True, palette=colors, size=2)
clean(ax, offset=4)
ax.set_ylabel('First Day Mean In-PF BSP')
ax.set_xlabel('Days')
ax.set_xticklabels(['1-2', '2-3'])

ax.get_legend().remove()

fig.savefig(os.path.join(plotdir, 'bsp_stability_by_day.svg'))

mannwhitneyu(df.loc[df['stable'] & (df['day_1']  == 1)]['value'], df.loc[~df['stable'] & (df['day_1']== 1)]['value'])
mannwhitneyu(df.loc[df['stable'] & (df['day_1']  == 2)]['value'], df.loc[~df['stable'] & (df['day_1']== 2)]['value'])

## SWR Index

In [ ]:
def pair_days(df, days=None):
    if days is None:
        days = [1, 2, 3]
    
    paired_dfs = []
    for d1 in days[:-1]:
        d2 = d1 + 1
        
        d1_df = df.loc[df['day'] == d1]
        d2_df = df.loc[df['day'] == d2]
        
        merged_df = d1_df.merge(d2_df, on=['mouse_name', 'roi', 'fov', 'condition'], suffixes=['_1', '_2'])
        merged_df['day'] = d1
        paired_dfs.append(merged_df)
    
    return pd.concat(paired_dfs)

paired_br = pair_days(running_br)

In [ ]:
running_br = pd.read_pickle(os.path.join(data_path, 'rf_running_br.pkl'))

In [ ]:
swr_br = pd.read_pickle(os.path.join(data_path, 'rf_swr_br.pkl'))

In [ ]:
merge_br = running_br.merge(swr_br, on=['cell', 'roi', 'mouse_name', 'fov', 'day', 'condition', 'expt'], suffixes=['_run', '_swr'])

In [ ]:
merge_br.dropna(subset=['value_run', 'value_swr'])
merge_br = merge_br.groupby(['cell', 'mouse_name', 'expt']).mean()
spearmanr(merge_br['value_run'], merge_br['value_swr'])

In [ ]:
nonrun_br = pd.read_pickle(os.path.join(data_path, 'rf_nonrunning_br.pkl'))

In [ ]:
# Alternate, stricter definitions
def group_dends(x):
    
    if (x['n_coactive'] > 0) and (x['n_solo'] == 0):
        return 'coactive'
    elif (x['n_solo'] > 0) and (x['n_coactive'] == 0):
        return 'solo'
    elif x['n_solo'] + x['n_coactive'] > 0:
        return 'mixed'
    elif x['n_solo'] + x['n_coactive'] == 0:
        return 'other'
    else:
        print 'WHAT'

def swr_coactivity_index(x):
    
    if x['n_coactive'] + x['n_solo'] == 0:
        return np.nan
    
    return (x['n_coactive'] - x['n_solo'])

def sci_grp(x):
    if x <= -1:
        return -1
    elif x >= 4:
        return 3
    else:
        return x

In [ ]:
swr_responses['type'] = swr_responses.apply(group_dends, axis=1)

In [ ]:
swr_responses['type'] = swr_responses.apply(group_dends, axis=1)
merge_df = swr_responses.merge(paired_br, on=['mouse_name', 'roi', 'fov', 'day', 'condition'], suffixes=['_swr', ''])
merge_df['delta'] = merge_df['value_2'] - merge_df['value_1']

merge_df['sci'] = merge_df.apply(swr_coactivity_index, axis=1)
merge_df['sci_grp'] = merge_df['sci'].apply(sci_grp)

fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['#00a79d', '#9e1f63']
sns.lineplot(x='sci_grp', y='delta', hue='day_1', data=merge_df, palette=colors, err_style='bars', ci=68, lw=1, err_kws={'elinewidth':1})

clean(ax, offset=4)
ax.set_ylabel(r'24H $\Delta$ Dendrite-Soma Coupling')
ax.set_xlabel('N Coactive - N Isolated\nSWR Events')

ax.axhline(0, color='k', ls='--', lw=1)

ax.set_xticks(range(-1, 4))
ax.set_xticklabels(['-1-', '0', '1', '2', '3+'])

ax.get_legend().set_visible(False)

ax.set_ylim([-0.2, 1])
# fig.savefig(os.path.join(plotdir, 'sci_by_condition.svg'))

In [ ]:
fig = plt.figure(figsize=(1.2, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['#ecd68e', '#f9b0a3', '#cccccc']

# Look at each day separately so we dont have interpretive issues (i.e. recruited then unrecruited)
# Though multi-day dynamics of recruitment would be interesting too
df = merge_df.loc[merge_df['day_1'] == 1]
df = df.loc[df['type'] != 'mixed']

sns.boxplot(x='type', y='delta', whis=[5, 95], data=df, order=['coactive', 'solo', 'other'],
            showcaps=True, showfliers=False, palette=colors)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='type', y='delta', data=df, order=['coactive', 'solo', 'other'],
              edgecolor='k', linewidth=0.5, alpha=0.5, palette=colors, size=2)

ax.set_ylabel(r'24H $\Delta$ Dendrite-Soma Coupling')
ax.set_xlabel('')
ax.set_xticklabels(['Coactive', 'Isolated', 'Unrecruited'])
ax.set_xlabel('Day 1 SWR-Related Activity')

clean(ax, offset=4)

plt.setp(ax.artists, edgecolor='k');
plt.setp(ax.lines, color='k');

ax.axhline(0, color='k', ls='--')

ax.set_ylim([-1.1, 1.1])

fig.savefig(os.path.join(plotdir, 'swr_delta_br_day1.svg'))

In [ ]:
fig = plt.figure(figsize=(1.2, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['#ecd68e', '#f9b0a3', '#cccccc']

# Look at each day separately so we dont have interpretive issues (i.e. recruited then unrecruited)
# Though multi-day dynamics of recruitment would be interesting too
df = merge_df.loc[merge_df['day_1'] == 2]
df = df.loc[df['type'] != 'mixed']

sns.boxplot(x='type', y='delta', whis=[5, 95], data=df, order=['coactive', 'solo', 'other'],
            showcaps=True, showfliers=False, palette=colors)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='type', y='delta', data=df, order=['coactive', 'solo', 'other'],
              edgecolor='k', linewidth=0.5, alpha=0.5, palette=colors, size=2)

ax.set_ylabel(r'24H $\Delta$ Dendrite-Soma Coupling')
ax.set_xlabel('')
ax.set_xticklabels(['Coactive', 'Isolated', 'Unrecruited'])
ax.set_xlabel('Day 2 SWR-Related Activity')

clean(ax, offset=4)

plt.setp(ax.artists, edgecolor='k');
plt.setp(ax.lines, color='k');

ax.axhline(0, color='k', ls='--')

# plt.xticks(rotation=45)
ax.set_ylim([-1.1, 1.1])

fig.savefig(os.path.join(plotdir, 'swr_delta_br_day2.svg'))

In [ ]:
# Control: Day 1

fig = plt.figure(figsize=(1.2, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['#ecd68e', '#f9b0a3', '#cccccc']

# Look at each day separately so we dont have interpretive issues (i.e. recruited then unrecruited)
# Though multi-dynamics of recruitment would be interesting too
df = merge_df.loc[merge_df['day_1'] == 1]
df = df.loc[df['type'] != 'mixed']

sns.boxplot(x='type', y='value_1', whis=[5, 95], data=df, order=['coactive', 'solo', 'other'],
            showcaps=True, showfliers=False, palette=colors)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='type', y='value_1', data=df, order=['coactive', 'solo', 'other'],
              edgecolor='k', linewidth=0.5, alpha=0.5, palette=colors, size=2)

# ax.set_ylabel(r'$\Delta$ RUN Branch Coincidence Rate')
ax.set_ylabel(r'Day 1 Dendrite-Soma Coupling')
ax.set_xlabel('')
ax.set_xticklabels(['Coactive', 'Isolated', 'Unrecruited'])
ax.set_xlabel('Day 1 SWR-Related Activity')

clean(ax, offset=4)

plt.setp(ax.artists, edgecolor='k');
plt.setp(ax.lines, color='k');

# ax.axhline(0, color='k', ls='--')

# plt.xticks(rotation=45)
ax.set_ylim([-0.1, 1.1])

fig.savefig(os.path.join(plotdir, 'swr_value_br_day1.svg'))

In [ ]:
a = df.loc[df['type'] == 'coactive']['value_1'].dropna()
b = df.loc[df['type'] == 'solo']['value_1'].dropna()
c = df.loc[df['type'] == 'other']['value_1'].dropna()

print f_oneway(a, b, c)

print a.shape[0], b.shape[0], c.shape[0]

print a.mean(), a.sem()
print b.mean(), b.sem()
print c.mean(), c.sem()

print pairwise_tukeyhsd(endog=df['value_1'], groups=df['type'], alpha=0.05)

In [ ]:
def swr_coactivity_normed(x):
    
    if x['n_coactive'] + x['n_solo'] == 0:
        return np.nan
    
    return (x['n_coactive'] - x['n_solo']) / float(x['n_coactive'] + x['n_solo'])

In [ ]:

swr_responses['type'] = swr_responses.apply(group_dends, axis=1)
merge_df = swr_responses.merge(paired_br, on=['mouse_name', 'roi', 'fov', 'day', 'condition'], suffixes=['_swr', ''])
merge_df['delta'] = merge_df['value_2'] - merge_df['value_1']

merge_df['scn'] = merge_df.apply(swr_coactivity_normed, axis=1)

fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['#00a79d', '#9e1f63']
# sns.lineplot(x='scn', y='delta', hue='day_1', data=merge_df, palette=colors, err_style='bars', ci=68, lw=1, err_kws={'elinewidth':1})
sns.scatterplot(x='scn', y='delta', hue='day_1', data=merge_df, palette=colors)

clean(ax, offset=4)
ax.set_ylabel(r'24H $\Delta$ Dendrite-Soma Coupling')
ax.set_xlabel('N Coactive - N Isolated\nN Coactive + N Isolated')

ax.axhline(0, color='k', ls='--')

# ax.set_xticks(range(-1, 4))
# ax.set_xticklabels(['-1-', '0', '1', '2', '3+'])

ax.get_legend().set_visible(False)

ax.set_ylim([-1.1, 1.1])
fig.savefig(os.path.join(plotdir, 'swr_index_by_day.svg'))

In [ ]:
merge_df = merge_df.dropna(subset=['scn', 'delta'])

In [ ]:
spearmanr(merge_df.loc[merge_df['day_1'] == 1]['scn'], merge_df.loc[merge_df['day_1'] == 1]['delta'])

In [ ]:
spearmanr(merge_df.loc[merge_df['day_1'] == 2]['scn'], merge_df.loc[merge_df['day_1'] == 2]['delta'])

In [ ]:
fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)
ax.hist(merge_df['scn'].dropna())
ax.set_xlabel('N Coactive - N Isolated\nN Coactive + N Isolated')
ax.set_ylabel('N Dendrites')
clean(ax)
fig.savefig(os.path.join(plotdir, 'sci_distribution.svg'))

In [ ]:
nonrun_branch_impact = da.branch_impact(grp, signal=None, non_running_only=True, non_ripple_only=True)

In [ ]:
# Same as above but for REST
nonrun_branch_impact['type'] = nonrun_branch_impact.apply(group_dends, axis=1)
merge_df = nonrun_branch_impact.merge(paired_br, on=['mouse_name', 'roi', 'fov', 'day', 'condition'], suffixes=['_swr', ''])
merge_df['delta'] = merge_df['value_2'] - merge_df['value_1']

merge_df['sci'] = merge_df.apply(swr_coactivity_normed, axis=1)

fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['#00a79d', '#9e1f63']
sns.scatterplot(x='sci', y='delta', hue='day_1', data=merge_df, palette=colors)

clean(ax, offset=4)
ax.set_ylabel(r'24H $\Delta$ Dendrite-Soma Coupling')
ax.set_xlabel('N Coactive - N Isolated\nN Coactive + N Isolated')

ax.get_legend().set_visible(False)

ax.set_ylim([-1.1, 1.1])
fig.savefig(os.path.join(plotdir, 'REST_index_by_condition.svg'))

print spearmanr(merge_df.loc[merge_df['day_1'] == 1]['sci'], merge_df.loc[merge_df['day_1'] == 1]['delta'])
print spearmanr(merge_df.loc[merge_df['day_1'] == 2]['sci'], merge_df.loc[merge_df['day_1'] == 2]['delta'])

In [ ]:
fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)
ax.hist(merge_df['sci'].dropna())

ax.set_xlabel('N Coactive - N Isolated\nN Coactive + N Isolated')
ax.set_ylabel('N Dendrites')
clean(ax)
fig.savefig(os.path.join(plotdir, 'sci_distribution_REST.svg'))

In [ ]:
fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['#ecd68e', '#f9b0a3', '#cccccc']

# Look at each day separately so we dont have interpretive issues (i.e. recruited then unrecruited)
# Though multi-day dynamics of recruitment would be interesting too
df = merge_df.loc[merge_df['day_1'] == 1]
df = df.loc[df['type'] != 'mixed']

sns.boxplot(x='type', y='delta', whis=[5, 95], data=df, order=['coactive', 'solo'],
            showcaps=True, showfliers=False, palette=colors)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='type', y='delta', data=df, order=['coactive', 'solo'],
              edgecolor='k', linewidth=0.5, alpha=0.5, palette=colors, size=2)

ax.set_ylabel(r'24H $\Delta$ Dendrite-Soma Coupling')
ax.set_xlabel('')
ax.set_xticklabels(['Coactive', 'Isolated'])
ax.set_xlabel('Day 1 REST-Related Activity')

clean(ax, offset=4)

plt.setp(ax.artists, edgecolor='k');
plt.setp(ax.lines, color='k');

ax.axhline(0, color='k', ls='--')

ax.set_ylim([-1.1, 1.1])

# fig.savefig(os.path.join(plotdir, 'REST_delta_br_day1.svg'))

In [ ]:
a = df.loc[df['type'] == 'coactive']['value_1'].dropna()
b = df.loc[df['type'] == 'solo']['value_1'].dropna()

print ttest_ind(a, b)

print a.shape[0], b.shape[0], c.shape[0]

print a.mean(), a.sem()
print b.mean(), b.sem()
print c.mean(), c.sem()

print pairwise_tukeyhsd(endog=df['value_1'], groups=df['type'], alpha=0.05)

## Compute

In [ ]:
data_path = '/home/sebi/data/dends'

In [ ]:
grp[0].get('tSeriesDirectory')

In [ ]:
def indicator(pf, nbins=100):
    
    pf_indicator = np.zeros((nbins,), dtype=bool)
    start = int(pf[0] * nbins)
    stop = int(pf[1] * nbins)
    if start < stop:
        pf_indicator[start:stop] = 1
    else:
        pf_indicator[start:] = 1
        pf_indicator[:stop] = 1
    return pf_indicator

def tc_correlation(cell_set):
    data_list = []
    for cell in cell_set:
        
        mouse_name = cell.parent_experiment.parent.mouse_name
        fov = cell.parent_experiment.get('uniqueLocationKey')
        day = cell.parent_experiment.get('day')
        condition = cell.parent_experiment.get('condition')
        session = cell.parent_experiment.get('session')
        tid = cell.parent_experiment.trial_id
        
        belt_len = cell.parent_experiment.belt_length / 1000.
        
        soma_tc = cell.soma.data()
        soma_peak = np.argmax(soma_tc)
        
        if cell.soma.pfs():
            soma_pf = True
            soma_indicator = indicator(cell.soma.pfs()[0])
        else:
            soma_pf = False
            soma_indicator = np.zeros((100,), dtype=bool)
        
        dend_tcs = [d.data() for d in cell.dendrites]
        
        for d, d_tc in zip(cell.dendrites, dend_tcs):
            
            corr = pearsonr(d_tc, soma_tc)[0]
            dend_peak = np.argmax(d_tc)
            
            peak_dist = dist(soma_peak, dend_peak)
            peak_dist *= belt_len
            
            if d.pfs():
                dend_pf = True
                dend_indicator = indicator(d.pfs()[0])
            else:
                dend_pf = False
                dend_indicator = np.zeros((100,), dtype=bool)
            
            pf_overlap = np.any(soma_indicator & dend_indicator)
            
            data_list.append({'mouse': mouse_name,
                              'fov': fov,
                              'day': day,
                              'condition': condition,
                              'session': session,
                              'expt': tid,
                              'soma': cell.soma.label,
                              'dend': d.label,
                              'order': d.order,
                              'corr': corr, 
                              'pf': soma_pf,
                              'dend_pf': dend_pf,
                              'pf_overlap': pf_overlap,
                              'peak_dist': peak_dist,
                              'abs_dist': np.abs(peak_dist)})

    return pd.DataFrame(data_list)

In [ ]:
cells = da.CellSet(grp)

tc_corrs = tc_correlation(cells)

In [ ]:
tc_corrs.to_pickle(os.path.join(data_path, 'tc_corrs.pkl'))

In [ ]:
def convert_centroid(x):
    """Convert centroid from complex value to position.

    Take angle and convert from radians.
    """
    return ((np.angle(x) % (2 * np.pi))) / (2 * np.pi) * 100

def dist(a, b):
    x = a - b
    if x <= -50:
        x += 100
    elif x > 50:
        x -= 100
    return x

def calc_activity_centroids(grp):
    soma_filter = lambda x: '_' not in x.label

    data_list = []

    centroids = pca.calc_activity_centroids(grp, roi_filter=soma_filter)

    pfs = grp.pfs(roi_filter=soma_filter)

    for expt in grp:

        trial_id = expt.trial_id
        mouse_name = expt.parent.mouse_name

        rois = expt.rois(roi_filter=soma_filter)

        for roi, pf, centroid in zip(rois, pfs[expt], centroids[expt]):
            if centroid:
                centroid_pos = convert_centroid(centroid)

                if pf:
                    is_pf = True
                else:
                    is_pf = False

                data_list.append({'mouse_name': mouse_name,
                                  'expt': trial_id,
                                  'session': expt.session,
                                  'condition': expt.condition,
                                  'day': expt.day,
                                  'roi': roi.label,
                                  'fov': expt.get('uniqueLocationKey'),
                                  'centroid': centroid_pos,
                                  'pf': is_pf})
                
    centroid_dists = pd.DataFrame(data_list).dropna()
    return centroid_dists

def calc_pf_centroids(grp):
    soma_filter = lambda x: '_' not in x.label

    data = grp.data(roi_filter=soma_filter)
    pfs = grp.pfs(roi_filter=soma_filter)

    data_list = []

    for expt in grp:

        centroids = pca.calcCentroids(data[expt], pfs[expt], returnAll=False)
        rois = expt.rois(label='mergedmerged', roi_filter=soma_filter)

        for roi, centroid in zip(rois, centroids):

            try:
                centroid = centroid[0]
            except IndexError:
                centroid = np.nan

            data_list.append({'expt': expt.trial_id,
                              'mouse_name': expt.parent.mouse_name,
                              'condition': expt.condition,
                              'day': expt.day,
                              'roi': roi.label,
                              'fov': expt.get('uniqueLocationKey'),
                              'centroid': centroid})

    centroids = pd.DataFrame(data_list)
    return centroids

def pair_days(df, days=None):
    if days is None:
        days = [1, 2, 3]
    
    paired_dfs = []
    for d1 in days[:-1]:
        d2 = d1 + 1
        
        d1_df = df.loc[df['day'] == d1]
        d2_df = df.loc[df['day'] == d2]
        
        merged_df = d1_df.merge(d2_df, on=['mouse_name', 'roi', 'fov', 'condition'], suffixes=['_1', '_2'])
        merged_df['day'] = d1
        paired_dfs.append(merged_df)
    
    return pd.concat(paired_dfs)

In [ ]:
activity_centroids = calc_activity_centroids(grp)
paired_activity_dists = pair_days(activity_centroids)
paired_activity_dists['absolute_centroid_shift'] = paired_activity_dists.apply(lambda x: np.abs(dist(x['centroid_1'], x['centroid_2'])) / 100., axis=1)
paired_activity_dists['expt'] = paired_activity_dists['expt_1']
activity_centroids.to_pickle(os.path.join(data_path, 'rf_activity_centroids.pkl'))
paired_activity_dists.to_pickle(os.path.join(data_path, 'rf_paired_activity_dists.pkl'))

In [ ]:
pf_centroids = calc_pf_centroids(grp)
paired_centroid_dists = pair_days(pf_centroids)
paired_centroid_dists['absolute_centroid_shift'] = paired_centroid_dists.apply(lambda x: np.abs(dist(x['centroid_1'], x['centroid_2'])) / 100., axis=1)
paired_centroid_dists['expt'] = paired_centroid_dists['expt_1']

pf_centroids.to_pickle(os.path.join(data_path, 'rf_pf_centroids.pkl'))
paired_centroid_dists.to_pickle(os.path.join(data_path, 'rf_paired_centroid_dists.pkl'))

In [ ]:
swr_responses = da.swr_activity(grp)

In [ ]:
swr_responses.to_pickle(os.path.join(data_path, 'rf_swr_responses.pkl'))

In [ ]:
# Properties of dendrites with isolated tuning
sens = pca.sensitivity(grp)
spef = pca.specificity(grp)
split_corr = pca.within_session_correlation(grp, n_splits=1)

sens = sens.rename(index=str, columns={'expt_id': 'expt', 'roi':'dend', 'FOV':'fov'})
spef = spef.rename(index=str, columns={'expt_id': 'expt', 'roi':'dend', 'FOV':'fov'})
split_corr = split_corr.rename(index=str, columns={'expt_id': 'expt', 'roi':'dend', 'FOV':'fov'})

In [ ]:
# Same as above but shift cells to be centered somatic peak'

soma_filter = lambda x: '_' not in x.label
dend_filter = lambda x: '_' in x.label

all_tcs = [[], [], []]
soma_data = grp.data(roi_filter=soma_filter)
soma_pfs = grp.pfs(roi_filter=soma_filter)

dend_data = grp.data(roi_filter=dend_filter)
dend_pfs = grp.pfs(roi_filter=dend_filter)
bad_dends = []
for expt in grp:
    
    soma_rois = expt.rois(roi_filter=soma_filter)
    soma_labels = [x.label.lower() for x in soma_rois]
    
    shift_dict = {}
    for soma_label, soma_tc in zip(soma_labels, soma_data[expt]):
        shift_dict[soma_label] = 50 - np.argmax(soma_tc)
    
    dend_rois = expt.rois(roi_filter=dend_filter)
    dend_labels = [x.label.lower() for x in dend_rois]
    
    expt_tcs = []
#     print expt.trial_id, expt.day, len([x for x in dend_pfs[expt] if x]), len(dend_pfs[expt])
    for dend_label, dend_tc, dend_pf in zip(dend_labels, dend_data[expt], dend_pfs[expt]):
        if not dend_pf:
            continue
        
        try:
            shift = shift_dict[dend_label.split('_')[0]]
        except KeyError:
            bad_dends.append((expt.trial_id, dend_label))
            continue
        
        expt_tcs.append(np.roll(dend_tc, shift))
    
    
    all_tcs[expt.day - 1].extend(expt_tcs)

fig = plt.figure()
for i in xrange(1, len(all_tcs)+1):
    ax = fig.add_subplot(1, len(all_tcs), i)
    
    if len(all_tcs[i-1]) == 0:
        continue
    
    hm = np.vstack(all_tcs[i-1])
    maxes = np.argmax(hm, axis=1)
    sort_idx = np.argsort(maxes)
    new_hm = hm[sort_idx, :]
    new_hm = new_hm / np.max(new_hm, axis=1, keepdims=True)
    
    print new_hm.shape
    
    sns.heatmap(new_hm, cbar=False, rasterized=True)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_yticklabels([0, new_hm.shape[0]-1], rotation=0)
    
    ax.axvline(50, color='w', ls='--')

# fig.savefig('/home/sebi/plots/dends/rf_all_tuned_dends_by_day_sorted_by_soma.svg')